In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

In [2]:
inp=pd.read_csv("../PJ sensor.csv",usecols=[6,7,10,11])
out=pd.read_csv("../PJ sensor.csv",usecols=[2,3,4,5,8,9])

In [3]:
inp.shape

(1627533, 4)

In [4]:
inp.head(5)

,Temperature,Relative Humidity,date,Time
0,31,65,191213,175844
1,31,65,191213,175904
2,31,65,191213,175924
3,31,65,191213,175944
4,31,66,191213,180004


In [5]:
out.head(5)

,NO2,O3,CO,SO2,PM2.5,PM10
0,-15.075377,35.714286,1766.233766,148.484848,11.229150,17.323938
1,-20.100503,41.208791,1733.766234,142.424242,9.413914,24.967407
2,-15.075377,43.956044,1711.038961,136.363636,10.350110,56.781536
3,0.000000,57.692308,1688.311688,139.393939,10.488765,30.371115
4,-20.100503,54.945055,1681.818182,130.303030,12.353048,188.026886


In [3]:
inp=np.array(inp)
out=np.array(out)

In [4]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(inp)
Y1=scaler_obj.fit_transform(out)

warnings.filterwarnings(action='ignore', category=UserWarning)

In [7]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.2,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(KNeighborsRegressor(n_neighbors=7,weights='uniform',algorithm='auto',leaf_size=50,p=2))

model_fit1=model1.fit(x_train, y_train)
print("Model training is Done!!")

filename1 = 'knn.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

Model training is Done!!


In [8]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')


r2 score on train data 0.8164847956978979
r2 score on test data 0.7463683061096252
Mean Absolute Error: 0.006902047365753544
Mean Squared Error: 0.0002061790700894036
Root Mean Squared Error: 0.014358936941480159
 



In [5]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("knn.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[0.47586207 0.60575595 0.04632433 0.00866075 0.08603813 0.0049559 ]
 [0.43743842 0.58059985 0.02749619 0.00823584 0.04023669 0.00283947]
 [0.4955665  0.60758549 0.06811287 0.01019699 0.10182398 0.00658002]
 ...
 [0.41674877 0.57602601 0.03727735 0.0106219  0.08458749 0.00652922]
 [0.41970443 0.57419648 0.03509849 0.0117986  0.08173987 0.0073757 ]
 [0.47192118 0.56276189 0.06081844 0.00866075 0.09352294 0.00856873]]


R2 Score :  0.7591639330678532


In [6]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=r2_score(y_test[:,i],y_test_pred[:,i])
    print("R2 Score of "+train_sizes[i]+":",temp)

R2 Score of NO2: 0.8093309816860448
R2 Score of O3: 0.629957641250577
R2 Score of NO: 0.8558734749957413
R2 Score of CO: 0.453829289834098
R2 Score of PM2.5: 0.8083098388422899
R2 Score of PM10: 0.25975872143425127


In [7]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=metrics.mean_squared_error(y_test[:,i],y_test_pred[:,i])
    print("MSE Value of "+train_sizes[i]+":",temp)

MSE Value of NO2: 0.00036505121017440763
MSE Value of O3: 0.0004950616640982264
MSE Value of NO: 2.76540638948933e-05
MSE Value of CO: 1.6557238738325072e-06
MSE Value of PM2.5: 0.00026942677466453935
MSE Value of PM10: 1.4669190395862972e-05


In [8]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=np.sqrt(metrics.mean_squared_error(y_test[:,i],y_test_pred[:,i]))
    print("RMSE Value of "+train_sizes[i]+":",temp)

RMSE Value of NO2: 0.01910631335905511
RMSE Value of O3: 0.02224998121568255
RMSE Value of NO: 0.00525871314057853
RMSE Value of CO: 0.0012867493438243934
RMSE Value of PM2.5: 0.016414224765871198
RMSE Value of PM10: 0.003830037910499447


In [9]:
train_sizes=['NO2','O3','NO','CO','PM2.5','PM10']
for i in range(0,6):
    temp=metrics.mean_absolute_error(y_test[:,i],y_test_pred[:,i])
    print("MAE Value of "+train_sizes[i]+":",temp)

MAE Value of NO2: 0.01251488807023571
MAE Value of O3: 0.012468864378297213
MAE Value of NO: 0.0020366124166244907
MAE Value of CO: 0.00047778097917830015
MAE Value of PM2.5: 0.010581534361050053
MAE Value of PM10: 0.002134438903783678


In [ ]:
#completed